In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pandas as pd
# import researchpy as rp
import scipy.stats as stats
import statsmodels.api as sm

from sklearn.model_selection import cross_val_score, StratifiedKFold, cross_val_predict, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import confusion_matrix,precision_recall_curve,auc,roc_auc_score,roc_curve,recall_score,classification_report,accuracy_score 
import itertools

# % matplotlib inline

accepted = pd.read_csv(
    '../input/lending-club/accepted_2007_to_2018q4.csv/accepted_2007_to_2018Q4.csv',
    parse_dates=['issue_d'], infer_datetime_format=True)
accepted = accepted[(accepted.issue_d >= '2017-01-01 00:00:00') & (accepted.issue_d < '2019-01-01 00:00:00')]
accepted = accepted.reset_index(drop=True)
accepted.head() 
df2=accepted[['funded_amnt','emp_length','zip_code','addr_state','dti','fico_range_high','purpose']]
df2.columns = ['Amount Requested', 'Employment Length', 'Zip Code', 'State', 'Debt-To-Income Ratio','Risk_Score','purpose']
df2=df2.drop(['purpose'], axis = 1) 
df2['accepted']=1

In [5]:
rejected = pd.read_csv(
    '../input/lending-club/rejected_2007_to_2018q4.csv/rejected_2007_to_2018Q4.csv')
# rejected = rejected[(rejected.issue_d >= '2017-01-01 00:00:00') & (rejected.issue_d < '2019-01-01 00:00:00')]
# rejected = rejected.reset_index(drop=True)
rejected.head() 
rejected=rejected.drop(['Loan Title'], axis = 1) 
rejected=rejected.drop(['Application Date'], axis = 1) 
rejected=rejected.drop(['Policy Code'], axis = 1) 
df1=rejected[['Amount Requested', 'Employment Length', 'Zip Code', 'State', 'Debt-To-Income Ratio','Risk_Score']]
df1['accepted']=0

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
df = pd.concat([df1,df2],ignore_index=True)
df['Debt-To-Income Ratio']=df['Debt-To-Income Ratio'].astype(str).map(lambda x: x.rstrip('%'))
df['Debt-To-Income Ratio']=df['Debt-To-Income Ratio'].astype(float)
df = df[df['Employment Length'].notna()]
df=df.drop(['Zip Code'], axis = 1) 
df = df.dropna(how='any',axis=0)


# q = df["Amount Requested"].quantile(0.99)
# df=df[df["Amount Requested"] < q]


In [12]:
#Determine Numeric & Categorical Columns
numeric=df.columns[df.dtypes=="float64"]
cat=df.columns[df.dtypes=="object"]

#Normalize numeric columns and create dummy variables with categorical columns

scaler=StandardScaler()
df[numeric] = scaler.fit_transform(df[numeric])
df=pd.get_dummies(df,drop_first=True)
X = df.drop('accepted', axis=1)
y = df['accepted']
X_train, X_test, y_train, y_test = train_test_split(X, y)



In [13]:
#TensorFlow Model 


import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
tf.compat.v1.disable_eager_execution()
import numpy as np

# Reset the graph for Tensorboard
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    


# Variables
n_inputs = X_train.shape[1]
n_hidden1 = 15
n_hidden2 = 5
n_outputs = 2

# Reset the tensorboard graph
reset_graph()


# Placeholders
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")


# Structure of the Neural Network
with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1",
                             activation=tf.nn.relu)
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                             activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")

# Cost Function
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y,
                                                     logits=logits) # Difference from logits and actual y values
    loss = tf.reduce_mean(xentropy) # Get the average of the loss for each instance.

# Gradient Descent
learning_rate = 0.01

with tf.name_scope("train"):
    optimization = tf.train.GradientDescentOptimizer(learning_rate) # Determine the level of steps in gradient descent process.
    training_op = optimization.minimize(loss) # Get the training set with parameters that obtain the minimum loss.

# Evaluation
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1) # Did the highest score of logit is equivalent to the actual value(returns booleans)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32)) # We want the mean of the loss for every instance.
    


# Init and Saver
init = tf.global_variables_initializer() # This variable initializes all variables.
saver = tf.train.Saver() # Saves the training set 


amnt_epochs = 5
batch_size = 100

with tf.Session() as sess:
    init.run()
    
    for epoch in range(amnt_epochs):
        epoch_loss = 0
        i=0
        while i < len(X_train):
            start = i
            end = i+batch_size
            batch_x = np.array(X_train[start:end])
            batch_y = np.array(y_train[start:end])

            _, c = sess.run([training_op, loss], feed_dict={X: batch_x,
                                              y: batch_y})
            epoch_loss += c
            i+=batch_size
        acc_train = accuracy.eval(feed_dict={X: batch_x, y: batch_y})
        acc_test = accuracy.eval(feed_dict={X: X_test, y:y_test})

        print(epoch+1, 'Train accuracy: ', acc_train, 'Test accuracy: ', acc_test, 'Loss: ', epoch_loss)

1 Train accuracy:  0.943662 Test accuracy:  0.94817793 Loss:  9414.440069729462
2 Train accuracy:  0.92957747 Test accuracy:  0.9491348 Loss:  8859.027611339465
3 Train accuracy:  0.92957747 Test accuracy:  0.94954914 Loss:  8765.774554077536
4 Train accuracy:  0.92957747 Test accuracy:  0.9497248 Loss:  8713.440914921463
5 Train accuracy:  0.92957747 Test accuracy:  0.94989604 Loss:  8682.39115839079


In [15]:
df.groupby(['accepted']).count()

,Amount Requested,Debt-To-Income Ratio,Risk_Score,Employment Length_10+ years,Employment Length_2 years,Employment Length_3 years,Employment Length_4 years,Employment Length_5 years,Employment Length_6 years,Employment Length_7 years,...,State_SD,State_TN,State_TX,State_UT,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY
accepted,,,,,,,,,,,,,,,,,,,,,
0,8992576,8992576,8992576,8992576,8992576,8992576,8992576,8992576,8992576,8992576,...,8992576,8992576,8992576,8992576,8992576,8992576,8992576,8992576,8992576,8992576
1,864852,864852,864852,864852,864852,864852,864852,864852,864852,864852,...,864852,864852,864852,864852,864852,864852,864852,864852,864852,864852
